## Loading GeoJSON from Geoserver WFS

In [ ]:
!pip install matplotlib geopandas folium numpy pandas matplotlib rasterio

In [ ]:
import requests
import geopandas as gpd
import folium


# Accept user input for the GeoServer URL and the typeName of the layer
geoserver_url = "https://geowebservices.stanford.edu/geoserver/druid/ows?"
type_name = "druid:vx572wx7854"

# Define the parameters for the WFS service
params = {
    'service': 'WFS',
    'version': '2.0.0',
    'request': 'GetFeature',
    'typeName': type_name,
    'outputFormat': 'application/json'
}

# Send a GET request to the GeoServer WFS service
response = requests.get(geoserver_url, params=params)

# Define the center of the map
center = [35.0, -120.0]

# Create the map
m = folium.Map(location=center, zoom_start=2)

# Load the features from the WFS service into a GeoDataFrame
gdf = gpd.read_file(response.text)


# Add the GeoDataFrame to the map as a GeoJson layer
folium.GeoJson(gdf).add_to(m)

# Display the map
m

## Loading GeoJSON from a STACKS URL

In [ ]:
import requests
import geopandas as gpd
import folium

# Define the GeoJSON URL
geojson_url = "https://stacks.stanford.edu/file/druid:vx812cc5548/stanford-vx572wx7854-geojson.geojson"

# Send a GET request to the GeoJSON URL
response = requests.get(geojson_url)

# Define the center of the map
center = [35.0, -120.0]

# Create the map
m = folium.Map(location=center, zoom_start=2)

# Load the features from the GeoJSON URL into a GeoDataFrame
gdf = gpd.read_file(response.text)

# Add the GeoDataFrame to the map as a GeoJson layer
folium.GeoJson(gdf).add_to(m)

# Display the map
m

## Loading COG from a STACKS URL

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Specify the path for Landsat TIF on AWS
fp = 'https://stacks.stanford.edu/file/druid%3Avq494qx9344/odm_orthophoto_COG_d.tif'

# See the profile
with rasterio.open(fp) as src:
    print(src.profile)
    
    %matplotlib inline
# Open the COG
with rasterio.open(fp) as src:
    # List of overviews from biggest to smallest
    oviews = src.overviews(1)

    # Retrieve the smallest thumbnail
    oview = oviews[-1]
    print('Decimation factor= {}'.format(oview))
    # NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
    thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

print('array type: ',type(thumbnail))
print(thumbnail)

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

In [ ]:
import folium
from osgeo import gdal

# Define the URL of the cloud optimized GeoTIFF
geotiff_url = "/vsicurl/https://stacks.stanford.edu/file/druid%3Avq494qx9344/odm_orthophoto_COG_d.tif"

# Open the GeoTIFF file using GDAL
dataset = gdal.Open(geotiff_url)

# Read the image data from the GeoTIFF
image = dataset.ReadAsArray()

# Define the center of the map
center = [dataset.GetGeoTransform()[3] + dataset.RasterYSize * dataset.GetGeoTransform()[5] / 2,
          dataset.GetGeoTransform()[0] + dataset.RasterXSize * dataset.GetGeoTransform()[1] / 2]

# Convert the image data to RGBA format
image = image.transpose(1, 2, 0)

# Create the map
m = folium.Map(location=center, zoom_start=10)

# Add the image overlay to the map
folium.raster_layers.ImageOverlay(
    image=image,
    bounds=[[dataset.GetGeoTransform()[3], dataset.GetGeoTransform()[0]],
            [dataset.GetGeoTransform()[3] + dataset.RasterYSize * dataset.GetGeoTransform()[5],
             dataset.GetGeoTransform()[0] + dataset.RasterXSize * dataset.GetGeoTransform()[1]]],
    colormap=lambda x: (0, 0, 0, x),
).add_to(m)

# Display the map
m